<a href="https://colab.research.google.com/github/ML-HW-SYS/a3-WDaugherty/blob/main/1_auto_conv1d_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1D Convolution on CPU

## 1. Set-up 

In [1]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
# with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
#     handle = f.readline().strip()

In [10]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = 'WDaugherty'
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-WDaugherty' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-WDaugherty
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 451 bytes | 15.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-WDaugherty
   ba3efdd..3a34abd  main       -> origin/main
Updating ba3efdd..3a34abd
Fast-forward
 src/ops.py | 8 +++++---
 1 file changed, 5 insertions(+), 3 deletions(-)
/content/gdrive/MyDrive/ece5545


In [5]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

Verify the following cell prints your github repository.

In [4]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb   4_gemm_gpu.ipynb	      space-time-dwsp.ipynb
1_conv1d_cpu.ipynb   5-conv2d_dw_gpu.ipynb    space-time-GEMM.ipynb
2-conv1d_gpu.ipynb   leaderboard_id.txt       src
2_conv1d_gpu.ipynb   README.md		      tests
3-conv1d_fpga.ipynb  space-time-1D_CPU.ipynb
4-gemm_gpu.ipynb     space-time-1D_GPU.ipynb


## 2. Install TVM

In [5]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://tlcpack.ai/wheels


## 3. Implement `make_conv1d_cpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then 
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_cpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array. 
You should return both the TVM schedule and the TVM operator for 
1. Input $x$
2. Input $y$
3. Output $out$

The schedule should be able to used to build a function with signature $func(x, y, out)$. 
Please see the following cells the usage.

In [13]:
import tvm
import numpy as np
import sys

import logging
import tvm
from tvm import te
import numpy as np
from tvm import autotvm
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_conv1d_cpu_scheduler



M = 4096
N = 128
dtype = 'float32'

task = autotvm.task.create("make_conv1d_cpu_scheduler", args=(M, N), target='llvm')

# Set the search space
n_trial = 1000
measure_option = autotvm.measure_option(
    builder=autotvm.LocalBuilder(),
    runner=autotvm.LocalRunner(number=20, repeat=3, min_repeat_ms=100, timeout=4)
)

tuner = autotvm.tuner.RandomTuner(task)
tuner.tune(n_trial=n_trial,
           measure_option=measure_option,
           callbacks=[autotvm.callback.log_to_file("conv1d_cpu.log")])

# Load the best configuration found by AutoTVM
dispatch_context = autotvm.apply_history_best("conv1d_cpu.log")
best_config = dispatch_context.query(task.target, task.workload)





a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

# Build the function using the best configuration
with tvm.target.Target('llvm'):
    with autotvm.apply_history_best("conv1d_cpu.log"):
        s, [A, W, B] = make_conv1d_cpu_scheduler(M, N)
        func = tvm.build(s, [A, W, B], "llvm")



dev = tvm.cpu()
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M+N-1), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat=1)

print("Answer:", b_np)
print("Output:", b)
print(f"1D conv TVM runtime: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [0.48065463 0.65568525 0.6645936  ... 1.0530987  0.6486484  0.26731107]
Output: [0.48065463 0.65568525 0.6645936  ... 1.0530987  0.6486484  0.26731107]
1D conv TVM runtime: 0.795371 ms


In [ ]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_cpu.py